In [1]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff xxhash pyyaml geopandas

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
! pip install --quiet git+https://github.com/nandanrao/facebook-python-business-sdk@pagination

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from environs import Env
from adopt.malaria import window
from adopt.facebook.state import CampaignState

env = Env()
env.read_env('.env-template', override=True)
state = CampaignState(env, window(4))

In [ ]:
[len(a['targeting']['excluded_geo_locations']['custom_locations']) for a in state.adsets]

In [ ]:
[len(a['targeting']['geo_locations']['regions']) for a in state.adsets]

'cities'

In [95]:
import pandas as pd
from adopt.malaria import load_cities
from adopt.marketing import *

def get_control_cities():
    treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
    cities = load_cities('output/cities.csv')

    controls = cities \
        .merge(treatment_assignment) \
        .pipe(lambda df: df[df.treatment == 0]) \
        .pipe(lambda df: df.assign(rad = df.rad + 1))

    return controls

def lookup_region(state):
    regions = {
        'Uttar Pradesh': {
            "country": "IN",
            "key": "1754",
            "name": "Uttar Pradesh"
        },
        'Jharkhand': {
            "country": "IN",
            "key": "1756",
            "name": "Jharkhand"
        },
        'Chhatisgarh': {
            "country": "IN",
            "key": "1755",
            "name": "Chhattisgarh"
        }
    }

    regions = {**regions, 'combined': [v for v in regions.values()]}

    return regions[state]


def create_city(t):
    return {
        TargetingGeoLocationCity.Field.key: t['key'],
        TargetingGeoLocationCity.Field.radius: 25.0,
        TargetingGeoLocationCity.Field.distance_unit: 'kilometer'
    }

def template_targeting(c: AdsetConf, targets):
    targeting = {}

    excluded_locs = [create_location(lat, lng, rad)
                     for lat, lng, rad in c.excluded_locs]

    targeting[Targeting.Field.excluded_geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        TargetingGeoLocation.Field.custom_locations: excluded_locs
    }

    if targets is None:
        key = TargetingGeoLocation.Field.regions
        locs = lookup_region(c.cluster.name)

    else: 
        key = TargetingGeoLocation.Field.cities
        locs = [create_city(t) for t in targets]
        

    targeting[Targeting.Field.geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        key: locs
    }

    return targeting

def make_template_adsets(env, state, base_name, combined=True, cities=None, cities_as_target=True):
    controls = get_control_cities()


    if combined:
        by_state = [(Cluster('combined', 'combined'), 
                     [Location(r.lat, r.lng, r.rad) 
                      for _, r in controls.iterrows()])]
    else:
        
        by_state = [(Cluster(st, st), [Location(r.lat, r.lng, r.rad) 
                                       for _, r in df.iterrows()])
                    for st, df in controls.groupby('state')]        


    targets = None
    additional_excludes = []

    if cities and cities_as_target:
        targets = cities

    elif cities and not cities_as_target:
        additional_excludes = cities


    confs = [AdsetConf(state.campaign, cl, None, lcs + additional_excludes, [], 
                       10000, 'PAUSED', env('FACEBOOK_INSTA_ID'), 
                       48, None, None, None)
             for cl, lcs in by_state]

    confs = [(c, template_targeting(c, targets)) for c in confs]

    instructions = [create_adset(f'{base_name}-{c.cluster.name}', c, t) for c,t in confs]    

    return instructions

In [22]:
cities = load_cities('output/cities.csv')
treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
cities = cities.merge(treatment_assignment)

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True)

control_cities = get_control_cities()

In [96]:
city_cities = [l[0] for name, l in city_locs]

In [ ]:
from adopt.facebook.update import GraphUpdater

instructions = make_template_adsets(env, state, 'ie-2-cities-template', True, city_cities, True)

updater = GraphUpdater(state)

for i in instructions:
    report = updater.execute(i)
    print(report)

In [76]:
from facebook_business.adobjects.targetingsearch import TargetingSearch


with open('outs/cities.txt') as f:
    cities = [l.strip() for l in f]


city_locs = [(city, TargetingSearch.search({'type': 'adgeolocation', 'q': city, 'location_types': ['city'], 'country_code': 'IN'})) 
             for city in cities]

In [94]:
from facebook_business.adobjects.targetinggeolocationcity import TargetingGeoLocationCity
 # 'country',
 # 'distance_unit',
 # 'key',
 # 'name',
 # 'radius',
 # 'region',
 # 'region_id'




{'key': '1035921', 'radius': 25.0, 'distance_unit': 'kilometer'}

[('Mumbai',
  <TargetingSearch> {
      "country_code": "IN",
      "country_name": "India",
      "key": "1035921",
      "name": "Mumbai",
      "region": "Maharashtra",
      "region_id": 1735,
      "supports_city": true,
      "supports_region": true,
      "type": "city"
  }),
 ('Delhi',
  <TargetingSearch> {
      "country_code": "IN",
      "country_name": "India",
      "key": "1023040",
      "name": "Delhi",
      "region": "Delhi",
      "region_id": 1728,
      "supports_city": true,
      "supports_region": true,
      "type": "city"
  }),
 ('Bangalore',
  <TargetingSearch> {
      "country_code": "IN",
      "country_name": "India",
      "key": "1017930",
      "name": "Bangalore",
      "region": "Karnataka",
      "region_id": 1738,
      "supports_city": true,
      "supports_region": true,
      "type": "city"
  }),
 ('Hyderabad',
  <TargetingSearch> {
      "country_code": "IN",
      "country_name": "India",
      "key": "1027234",
      "name": "Hyderabad",
     

In [109]:
import geopandas as gpd

gdf = gpd.read_file('outs/clusters.shp').drop(columns=['treatment'])

gdf = gdf.drop(columns=['st_areasha']).merge(districts[['distcode', 'treatment']], how='left')

In [116]:
gdf.to_file('outs/clusters.shp')